# Alguns arquivos de salinidade em situ vieram corrompidos, para não perder dados, aqui está o processo que estou fazendo para ajeitar o aquivo

##1. Importando bibliotecas necessárias

In [1]:
import subprocess
import netCDF4
import pandas as pd

###1.1 Crie as pastas de apoio

In [73]:
subprocess.run(['mkdir','/content/Passo1'])
subprocess.run(['mkdir','/content/Passo2'])
subprocess.run(['mkdir','/content/Passo3'])

CompletedProcess(args=['mkdir', '/content/Passo3'], returncode=1)

###1.2 Extrair o arquivo comprimido do drive para o colab


In [154]:
subprocess.run(['unrar','x','/content/drive/MyDrive/2006.rar'])

CompletedProcess(args=['unrar', 'x', '/content/drive/MyDrive/2006.rar'], returncode=0)

##2. Primeiro vou converter os csv para arquivos que podem ser lidos no excel, trocando a (,) por (;) e o (.) por (,)

###2.1 Resgantando todos os arquivos e os tranformando em um array de nome de arquivos

In [156]:
files = subprocess.getoutput(['ls','/content/'])
files = files.split()
files.pop()
files.pop()
files.pop()
files.pop()
files.pop()

'drive'

###2.2 Fazendo a mudança de (,) para (;)

In [157]:
for fi in files:
  fin = open(fi, "rt")
  fout = open('/content/Passo1/'+fi, "wt")
  for line in fin:
	    fout.write(line.replace(',', ';'))
fin.close()
fout.close()

###2.2 Fazendo a mudança de (.) para (,)

In [158]:
for fi in files:
  fin = open('/content/Passo1/'+fi, "rt")
  fout = open('/content/Passo2/'+fi, "wt")
  for line in fin:
	    fout.write(line.replace('.', ','))
fin.close()
fout.close()

###2.3. O grande problema dos arquivos e que a maioria explode o numero de variaveis, para isso, vou acrescentar 7 (;) no final do arquivo para que ele junte de forma correta todas as variaveis

In [159]:
primeiraLinha = 0
for fi in files:
  primeiraLinha = 0
  fin = open('/content/Passo2/'+fi, "rt")
  fout = open('/content/Passo3/'+fi, "wt")
  for line in fin:
    if primeiraLinha == 0:
	    fout.write(line.replace('\n', ';;;;;;;\n'))
    else:
      fout.write(line)
    primeiraLinha = primeiraLinha+1
fin.close()
fout.close()

##3. Feito isso, carregue todos os documentos para podermos criar nosso df pandas unico do ano


> OBS: Não se preocupe com arquivos em branco ou repetidos, com um dataFrame pandas a gnt consegue lidar com isso na parte de pré processamento

> Parte 1: Agora crie um unico arquivo para todos os anos apenas com as colunas necessárias, caso não tenha a coluna, ele vai pular o df





In [160]:
dx = pd.DataFrame(data=None)
for fi in files:
  try:
    df = pd.read_csv('/content/Passo3/'+fi,
                   sep=';',
                   engine='python',
                   usecols=[
                            'DATE (YYYY-MM-DDTHH:MI:SSZ)','LATITUDE (degree_north)',
                            'LONGITUDE (degree_east)','PSAL (psu)','PRES (decibar)'])
    dx = pd.concat([dx,df])
  except: 
    pass

> Parte 2: Veja se ta tudo ok

In [163]:
dx

,DATE (YYYY-MM-DDTHH:MI:SSZ),LATITUDE (degree_north),LONGITUDE (degree_east),PRES (decibar),PSAL (psu)
0,2006-01-07T02:59:13Z,"-12,0400","-26,4920",10,"36,736"
1,2006-01-07T02:59:13Z,"-12,0400","-26,4920",20,"36,728"
2,2006-01-07T02:59:13Z,"-12,0400","-26,4920",30,"36,732"
3,2006-01-07T02:59:13Z,"-12,0400","-26,4920",40,"36,73"
4,2006-01-07T02:59:13Z,"-12,0400","-26,4920",50,"36,836"
...,...,...,...,...,...
496,2006-06-23T10:53:22Z,"39,1260","-29,8210","1329,2","35,097"
497,2006-06-23T10:53:22Z,"39,1260","-29,8210","1379,8","35,066"
498,2006-06-23T10:53:22Z,"39,1260","-29,8210","1428,2","35,066"
499,2006-06-23T10:53:22Z,"39,1260","-29,8210","1479,8","35,063"


> Parte 3: Agora salve o dataFrame

In [164]:
x = [dx['DATE (YYYY-MM-DDTHH:MI:SSZ)'][0:1]]
ano = x[0][0][0:4]
dx.to_csv('/content/drive/MyDrive/'+ano+'.csv',sep=';')

##4. Por ultimo, limpe as pastas

In [153]:
for fi in files:
   subprocess.run(['rm','/content/'+fi])
   subprocess.run(['rm','/content/Passo1/'+fi])
   subprocess.run(['rm','/content/Passo2/'+fi])
   subprocess.run(['rm','/content/Passo3/'+fi])